In [6]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ["LANGCHAIN_TRACING_V2"] = "true"


# Test with Ollama

In [7]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3:instruct")
# Test
llm.invoke("Traduce al español: 'Hello World, I am working with langchain'")

'¡Hola Mundo, estoy trabajando con Langchain!\n\n(Note: "Hello World" es un saludo tradicional en programación y no necesariamente se refiere a la empresa Langchain, pero como contexto me parece que es lo más probable)'

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un cómico español muy gracisos que cuenta chistes a los niños."),
    ("user", "{input}")
])
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input":"Cuenta un chiste de Jaimito"})

'¡Hola, amigos! ¡Estoy aquí para hacer reír con mis chistes más locos! ¡Vamos a empezar!\n\n¿Sabéis quién es el personaje más listo del mundo? ¡Jaimito, por supuesto! Y aquí tenéis un chiste de él:\n\nPor qué Jaimito no podía dormir...\n\n¡Es porque estaba preocupado por la próxima fiesta de cumpleaños de su abuela!\n\n¿Por qué?\n\n¡Porque iba a tener que bailar el polka con ella!\n\n¡Ja, ja, ja! ¡Eso es un chiste de Jaimito, amigos! ¿Qué os ha parecido?'

# Retrieval Chain

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


In [13]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Based on the provided context, Langsmith can help with testing by allowing you to "visualize test results".'

## With vector db retrieval

In [14]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

Based on the provided context, LangSmith can help with testing in several ways:

1. **Tracing**: LangSmith allows developers to create datasets for their LLM applications and use these to run tests on their applications. This enables them to evaluate their application's performance across multiple turns.
2. **Comparison View**: Langsmith provides a comparison view that allows developers to track and diagnose regressions in test scores across different revisions of their application.
3. **Playground**: The playground environment enables rapid iteration and experimentation, allowing developers to quickly test out different prompts and models.
4. **Beta Testing**: LangSmith supports beta testing by collecting data on how the LLM application is performing in real-world scenarios and providing feedback collection and run annotation capabilities.
5. **Automations**: Automations allow developers to perform actions on traces in near real-time, such as automatically scoring traces or sending th

# Conversation Retrieval Chain

In [15]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage


# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [16]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them somethin

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [18]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fi

# Agent

In [19]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [21]:
tools = [retriever_tool, search]

In [22]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


LangSmith User Guide | 🦜️🛠️ LangSmith

Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing​Beta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback​When launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and hi

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith can help with testing in various ways, especially in the context of LLM application development. Here are some ways LangSmith can assist with testing:\n\n1. **Beta Testing**: LangSmith allows developers to collect data on how their LLM applications are performing in real-world scenarios during beta testing. It helps in understanding the types of inputs the app is performing well or poorly on and tracking regressions/improvements.\n\n2. **Capturing Feedback**: When launching an application to initial users, LangSmith enables gathering human feedback on the responses it produces. This feedback helps in highlighting edge cases causing problematic responses and improving the application.\n\n3. **Annotating Traces**: LangSmith supports sending runs to annotation queues, where annotators can closely inspect interesting traces and annotate them with different criteria. This helps catch regressions across important evaluat